# Apply ETAD to a raw Sentinel-1 SLC

In [ ]:
import s1etad
from s1etad import Sentinel1Etad, ECorrectionType
import os
import zipfile
import tarfile
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime
import requests
import asf_search as asf
from scipy import constants
from s1etad_tools.cli.slc_correct import s1etad_slc_correct_main
import time

# Set the download folders

In [ ]:
SLC_folder = 'data/SLC/original'
Corrected_SLC_folder = 'data/SLC/corrected'
ETAD_folder = 'data/ETAD'
os.makedirs(SLC_folder, exist_ok=True)
os.makedirs(Corrected_SLC_folder, exist_ok=True)
os.makedirs(ETAD_folder, exist_ok=True)

# Set credentials

In [ ]:
# Earthdata - https://urs.earthdata.nasa.gov/users/new
earthdata_uid = ''
earthdata_pswd = ''

# Copernicus dataspace - https://dataspace.copernicus.eu/
copernicus_uid = ''
copernicus_pswd = ''

# Seach and download a SLC from ASF

In [ ]:
# A point over the SURAT basin 
wkt = "POINT (150.5759  -27.0252)"
prod = 'SLC'
mode = 'IW'
start_date = '2023-11-18T00:00:00Z'
end_date = '2023-11-20T00:00:00Z'

results = asf.search(platform=[asf.PLATFORM.SENTINEL1], 
                     intersectsWith=wkt, 
                     maxResults=10, 
                     processingLevel=prod,
                     beamMode=mode,
                     start=start_date,
                     end=end_date)
print(f'number of results: {len(results)}')
results[0].properties

In [ ]:
# initialise a session to download
session = asf.ASFSession()
session.auth_with_creds(earthdata_uid,earthdata_pswd)

In [ ]:
# download the first result
download = True
download_slc = results[0]
if download:
    slc_path = os.path.join(SLC_folder, download_slc.properties['fileName'])
    scene = download_slc.properties['sceneName']
    if not os.path.exists(slc_path):
        download_slc.download(path=SLC_folder, session=session)
    else:
        print(f'slc already downloaded : {slc_path}')

# Search and download ETAD file from copernicus dataspace
note at the time of writing ETAD coverage is limited to mid-2023 onwards

In [ ]:
def search_download_scene_etad(scene: str, username: str, password: str, etad_dir: str = '', unzip=True):
    """search and download an ETAD product for a corresponding scene. 
        see - https://documentation.dataspace.copernicus.eu/APIs/OData.html

    Args:
        scene (str): scene of interest. e.g. S1A_IW_SLC__1SSH_20231119T083317_20231119T083345_051283_062FEC_0B2C
        etad_dir (str): where to save the downloaded product
        username (str): username for the copernicus dataspace
        password (str): password for the copernicus dataspace
    Returns:
        etad_path : path to the downloaded ETAD product. None if a product was not found.
    """

    sat, mode, prod,_, pol, start, finish = scene.split('_')[:7]

    print(f'Searching Copernicus Dataspace for ETAD file...')
    # find the ETAD using the start and end timestamps from the SLC
    search_results = requests.get(
        f"https://catalogue.dataspace.copernicus.eu/odata/v1/Products?$filter=contains(Name,'ETA') and contains(Name,'{start}') and contains(Name,'{finish}')&$orderby=ContentDate/Start&$top=100"
        ).json()['value']
    files = [res['Name'] for res in search_results]
    print(f'ETAD files found {files}')
    assert len(search_results) == 1, "error. more than one ETAD product found for scene"
    prod_id = search_results[0]['Id']
    prod_name = search_results[0]['Name']
    etad_filename = prod_name + '.zip'
    
    # get a token from copernicus hub to enable download
    data = {
        'grant_type': 'password',
        'username': f'{username}',
        'password': f'{password}',
        'client_id': 'cdse-public',
    }
    response = requests.post(
        'https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token', data=data)
    access_token = response.json()['access_token']

    # download the ETAD product
    url = f"https://zipper.dataspace.copernicus.eu/odata/v1/Products({prod_id})/$value"
    headers = {"Authorization": f"Bearer {access_token}"}
    session = requests.Session()
    session.headers.update(headers)
    response = session.get(url, headers=headers, stream=True)
    etad_path = os.path.join(etad_dir, etad_filename)
    
    print(f'Downloding ETAD to : {etad_path}')
    if not os.path.exists(etad_path):
        with open(f"{etad_path}", "wb") as file:
            for chunk in response.iter_content(chunk_size=8192):
                if chunk:
                    file.write(chunk)
    else:
        print(f'ETAD already downloaded')

    if unzip:
        etad_safe = etad_path.replace('.zip', '')
        print(f'Unzipping to : {etad_safe}')
        if not os.path.isdir(etad_safe):
            archive = zipfile.ZipFile(etad_path, 'r')
            archive.extractall(etad_dir)
            archive.close()

    return etad_path if not unzip else etad_safe

In [ ]:
# download the ETAD
etad_path = search_download_scene_etad(
    scene,
    username=copernicus_uid,
    password=copernicus_pswd,
    etad_dir=ETAD_folder,
    unzip=True
)

# Investigate the ETAD
See more at https://s1etad.readthedocs.io/en/latest/index.html

In [ ]:
eta = Sentinel1Etad(etad_path)
swath = eta['IW1']
burst = swath[1]
correction_types = list(ECorrectionType.__members__.keys())
s1etad.ECorrectionType.__members__

# Get merged burst

In [ ]:
download_slc.properties

In [ ]:
# get timestamps from the scene
sat, mode, prod, _, pol, start, finish = scene.split('_')[0:7]
first_time = datetime.strptime(start, "%Y%m%dT%H%M%S")
last_time =  datetime.strptime(finish, "%Y%m%dT%H%M%S")
# query the catalogue for a subset of the swaths
df = eta.query_burst(first_time=first_time, last_time=last_time, product_name=slc_path)
df

In [ ]:
# set soome constants
dy = eta.grid_spacing['y']
dx = eta.grid_sampling['x'] * constants.c / 2
nswaths = len(df.swathID.unique())
vg = eta.grid_spacing['y'] / eta.grid_sampling['y']
vmin = 2.5
vmax = 3.5
to_km = 1. / 1000

In [ ]:
# plot the different correction layers
for cor in correction_types:
    print(cor)
    fig, ax = plt.subplots(figsize=[13, 7])
    merged_correction = eta.merge_correction(ECorrectionType.__dict__[cor],
                                            selection=df, meter=True)
    print(merged_correction.keys())
    var_ = 'x' if 'x' in merged_correction.keys() else 'y'
    merged_correction_data = merged_correction[var_]
    vmin = np.nanmin(merged_correction_data)
    vmax = np.nanmax(merged_correction_data)

    ysize, xsize = merged_correction_data.shape
    x_axis = np.arange(xsize) * dx * to_km
    y_axis = np.arange(ysize) * dy * to_km

    extent=[x_axis[0], x_axis[-1], y_axis[0], y_axis[-1]]
    im = ax.imshow(merged_correction_data, origin='lower', extent=extent,
                vmin=vmin, vmax=vmax, aspect='equal')

    ax.set_xlabel('Slant Range [km]')
    ax.set_ylabel('Azimuth [km]')

    name = merged_correction['name']
    unit = merged_correction['unit']

    ax.set_title(f'Merged swaths for {name} correction ({var_})')
    fig.colorbar(im, ax=ax, label=f'[{unit}]')

# Apply ETAD corrections to a SLC

In [ ]:
# application of correction from https://github.com/SAR-ARD/S1_NRB/blob/main/S1_NRB/etad.py
def find_etad_file(scene, ETAD_dir):
    """_summary_

    Args:
        scene (str): scene. e.g. S1A_IW_SLC__1SSH_20231119T083317_20231119T083345_051283_062FEC_0B2C
        ETAD_dir (str): locally accessible directory containing downloaded ETAD products

    Returns:
        _type_: _description_
    """
    ETAD_file = None
    sat, mode, prod,_, pol, start, finish = scene.split('_')[:7]
    print(f'Searching local directory for ETAD product : {ETAD_dir}')
    for f in os.listdir(ETAD_dir):
        if all(substring in f for substring in [sat, mode, pol[-2:], start, finish]):
            ETAD_file = f
            print(f'ETAD found : {ETAD_file}')
            break
    if ETAD_file is None:
        print('ETAD file not found')
    return ETAD_file

def process(slc_path: str, ETAD_file: str, out_dir: str, nthreads: int=4):
    """
    Apply ETAD correction to a Sentinel-1 SLC product.
    
    Parameters
    ----------
    slc_path: str
        The path to the Sentinel-1 SLC.
    etad_dir: str
        The directory containing ETAD products. This will be searched for products using the SLC.
    out_dir: str
        The directory to store results. An unzipped SAFE folder structure is created.
    nthreads: the number of threads
        The number of threads used for processing=

    Returns
    -------
    str
        path to the corrected SLC SAFE product.
    """
    # unzip the slc if it is not already
    if slc_path.endswith('.zip'):
        slc_zip = slc_path
        slc_folder = os.path.dirname(slc_zip)
        slc_path = slc_zip.replace(".zip",".SAFE")
        if not os.path.exists(slc_path): 
            print(f'unzipping slc to : {slc_path}')  
            with zipfile.ZipFile(slc_zip, 'r') as zip_ref:
                zip_ref.extractall(slc_folder)
    
    os.makedirs(out_dir, exist_ok=True)
    slc_base = os.path.basename(slc_path).replace('.zip', '.SAFE')
    slc_corrected = os.path.join(out_dir, slc_base)
    if not os.path.isdir(slc_corrected):
        start_time = time.time()
        ext = os.path.splitext(ETAD_file)[1]
        if ext in ['.tar', '.zip']:
            etad_base = os.path.basename(ETAD_file).replace(ext, '.SAFE')
            etad = os.path.join(out_dir, etad_base)
            if not os.path.isdir(etad):
                if ext == '.tar':
                    archive = tarfile.open(ETAD_file, 'r')
                else:
                    archive = zipfile.ZipFile(ETAD_file, 'r')
                archive.extractall(out_dir)
                archive.close()
        elif ext == '.SAFE':
            etad = ETAD_file
        else:
            raise RuntimeError('ETAD products are required to be .tar/.zip archives or .SAFE folders')
        print('Correcting SLC with ETAD product')
        s1etad_slc_correct_main(s1_product=slc_path,
                                etad_product=etad,
                                outdir=out_dir,
                                nthreads=nthreads,
                                order=0)  # using the default 1 introduces a bias of about -0.5 dB.
        t = round((time.time() - start_time), 2)
        print(f'Time taken: {t}')
    else:
        print(f'ETAD corrected product already exists: {slc_corrected}')
    return slc_corrected

In [ ]:
slc_corrected = process(slc_path, etad_path, out_dir=Corrected_SLC_folder)